In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pickle
import Plot
import model
import numpy as np
import pandas as pd

# Get lambdas

In [53]:
def generate_lambdas(size):
    weights = []
    # 36 weight vectors
    for i in range(1,9):
        for j in range(i+1,10):
            weights.append([i,j-i,10-j])
    res = []
    for i in range(int(size/2)):
        res.append(weights[i])
        res.append(weights[-i-1])
    res = np.array(res)
    indx = np.lexsort((res[:,2],res[:,0],res[:,1]))
    return (res[indx])/10

lambdas = generate_lambdas(20)

In [54]:
lambdas

array([[0.1, 0.1, 0.8],
       [0.2, 0.1, 0.7],
       [0.5, 0.1, 0.4],
       [0.6, 0.1, 0.3],
       [0.7, 0.1, 0.2],
       [0.8, 0.1, 0.1],
       [0.1, 0.2, 0.7],
       [0.2, 0.2, 0.6],
       [0.5, 0.2, 0.3],
       [0.6, 0.2, 0.2],
       [0.7, 0.2, 0.1],
       [0.1, 0.3, 0.6],
       [0.5, 0.3, 0.2],
       [0.6, 0.3, 0.1],
       [0.1, 0.4, 0.5],
       [0.5, 0.4, 0.1],
       [0.1, 0.5, 0.4],
       [0.1, 0.6, 0.3],
       [0.1, 0.7, 0.2],
       [0.1, 0.8, 0.1]])

# Compute Objective functions

In [29]:
def compute_objective_functions(solution, sensors_positions, sink_nodes_positions):
    f = [0,0,0] 
    for i in range(len(solution)):
        if(solution[i][0]==1):
            f[0] += solution[i][1]**2
            f[1] += 1

            distance = np.sqrt((sensors_positions[i][0]-sink_nodes_positions[0][0])**2 + (sensors_positions[i][1]-sink_nodes_positions[0][1])**2)
        
            f[2] += distance
            
    f[2]/=f[1]
    return f

In [85]:
# Contains first solution of every run times of every datasets
df_first = pd.DataFrame(columns=['Dataset No.','Run No.','Power weight','Num. weight','Dist. weight','Power','Num. Active Sensors','Avg. Dist. to Sink node'])
# Contains last solution of every run times of every datasets
df_last = pd.DataFrame(columns=['Dataset No.','Run No.','Power weight','Num. weight','Dist. weight','Power','Num. Active Sensors','Avg. Dist. to Sink node'])

In [86]:
num_dataset = 5
for dataset in range(num_dataset):
    with open(f'Datasets/uniform/50x1000unit/100sensors/sensors_positions_{dataset}.pickle','rb') as file:
        sensors_positions = pickle.load(file)
    with open('Datasets/sink_nodes_positions.pickle','rb') as file:
        sink_nodes_positions = pickle.load(file) 

    for runtime in range(5):
        with open(f'Results/uniform/50x1000unit/100sensors/dataset_{dataset}/last_solutions_{runtime}.pickle','rb') as file:
            last_sol = pickle.load(file)
        with open(f'Results/uniform/50x1000unit/100sensors/dataset_{dataset}/first_solutions_{runtime}.pickle','rb') as file:
            first_sol = pickle.load(file)
        
        for i in range(len(first_sol)):
            first_sol_f = compute_objective_functions(first_sol[i], sensors_positions, sink_nodes_positions)
            last_sol_f = compute_objective_functions(last_sol[i], sensors_positions, sink_nodes_positions)
            df_first.loc[len(df_first)] = [dataset, runtime, lambdas[i][0],lambdas[i][1],lambdas[i][2], first_sol_f[0], first_sol_f[1], first_sol_f[2]]
            df_last.loc[len(df_last)] = [dataset, runtime, lambdas[i][0],lambdas[i][1],lambdas[i][2], last_sol_f[0], last_sol_f[1], last_sol_f[2]]

In [71]:
df_first.iloc[-20:]

,Dataset No.,Run No.,Power weight,Num. weight,Dist. weight,Power,Num. Active Sensors,Avg. Dist. to Sink node
380,3.0,4.0,0.1,0.1,0.8,16060.336189,31.0,815.629296
381,3.0,4.0,0.2,0.1,0.7,16060.336189,31.0,815.629296
382,3.0,4.0,0.5,0.1,0.4,16060.336189,31.0,815.629296
383,3.0,4.0,0.6,0.1,0.3,16060.336189,31.0,815.629296
384,3.0,4.0,0.7,0.1,0.2,16060.336189,31.0,815.629296
385,3.0,4.0,0.8,0.1,0.1,16060.336189,31.0,815.629296
386,3.0,4.0,0.1,0.2,0.7,16060.336189,31.0,815.629296
387,3.0,4.0,0.2,0.2,0.6,16060.336189,31.0,815.629296
388,3.0,4.0,0.5,0.2,0.3,16060.336189,31.0,815.629296
389,3.0,4.0,0.6,0.2,0.2,16060.336189,31.0,815.629296


In [66]:
df_last.iloc[-20:].sort_values(by=['Num. Active Sensors'])

,Dataset No.,Run No.,Lambda,Power,Num. Active Sensors,Avg. Dist. to Sink node
380,3,4,"[0.1, 0.1, 0.8]",47895.283043,13,611.398495
391,3,4,"[0.1, 0.3, 0.6]",35904.937501,16,623.863534
386,3,4,"[0.1, 0.2, 0.7]",39881.987629,16,610.794251
381,3,4,"[0.2, 0.1, 0.7]",22689.050275,19,657.292987
396,3,4,"[0.1, 0.5, 0.4]",25931.996867,19,652.702392
394,3,4,"[0.1, 0.4, 0.5]",31627.740388,19,632.931459
399,3,4,"[0.1, 0.8, 0.1]",15825.964437,20,776.563826
387,3,4,"[0.2, 0.2, 0.6]",19836.520581,21,665.634121
397,3,4,"[0.1, 0.6, 0.3]",20075.902009,21,676.900421
398,3,4,"[0.1, 0.7, 0.2]",15109.266679,22,767.312412


# Join table by Dataset No., Run No., Lambdas

In [87]:
df_join =pd.merge(df_first,df_last,on=['Dataset No.','Run No.','Power weight','Num. weight','Dist. weight'],how='outer', suffixes=('_first','_last'))

In [91]:
df_join.to_csv('Results/uniform/50x1000unit/100sensors/First_vs_Last_solutions.csv')

In [92]:
df_join

,Dataset No.,Run No.,Power weight,Num. weight,Dist. weight,Power_first,Num. Active Sensors_first,Avg. Dist. to Sink node_first,Power_last,Num. Active Sensors_last,Avg. Dist. to Sink node_last
0,0.0,0.0,0.1,0.1,0.8,14270.961861,37.0,732.712838,16380.949927,20.0,753.958053
1,0.0,0.0,0.2,0.1,0.7,14270.961861,37.0,732.712838,15396.640337,21.0,730.086076
2,0.0,0.0,0.5,0.1,0.4,14270.961861,37.0,732.712838,19264.890446,23.0,646.687621
3,0.0,0.0,0.6,0.1,0.3,14270.961861,37.0,732.712838,24640.617885,21.0,629.700808
4,0.0,0.0,0.7,0.1,0.2,14270.961861,37.0,732.712838,8277.837825,41.0,743.879095
...,...,...,...,...,...,...,...,...,...,...,...
495,4.0,4.0,0.5,0.4,0.1,19765.960569,27.0,758.969072,8932.563623,38.0,768.756487
496,4.0,4.0,0.1,0.5,0.4,19765.960569,27.0,758.969072,26165.374987,16.0,665.574734
497,4.0,4.0,0.1,0.6,0.3,19765.960569,27.0,758.969072,19995.661268,18.0,680.391519
498,4.0,4.0,0.1,0.7,0.2,19765.960569,27.0,758.969072,15388.315305,21.0,762.413487
